{/* cspell:ignore mactex, backgroundcolor, lightgreen */}

# Visualize circuit timing

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.1.1
```
</details>

In addition to [visualizing instructions on a circuit](/docs/guides/visualize-circuits) you might want to use the Qiskit [`timeline_drawer`](/docs/api/qiskit/qiskit.visualization.timeline_drawer) method, which lets you visualize a circuit's scheduling. This visualization could help you to quickly spot idling time on qubits, for example.

## Examples

To visualize a scheduled circuit program, you can call this function with a set of control arguments. Most of the  output image's appearance can be modified by a stylesheet, but this is not required.

### Draw with the default stylesheet

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization.timeline import draw
from qiskit.providers.fake_provider import GenericBackendV2

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

backend = GenericBackendV2(5)

qc = transpile(qc, backend, scheduling_method="alap", layout_method="trivial")
draw(qc, target=backend.target)

### Draw with a simple stylesheet

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization.timeline import draw, IQXSimple
from qiskit.providers.fake_provider import GenericBackendV2

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

backend = GenericBackendV2(5)

qc = transpile(qc, backend, scheduling_method="alap", layout_method="trivial")
draw(qc, style=IQXSimple(), target=backend.target)

### Draw with a stylesheet suited for program debugging

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization.timeline import draw, IQXDebugging
from qiskit.providers.fake_provider import GenericBackendV2

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

backend = GenericBackendV2(5)
qc = transpile(qc, backend, scheduling_method="alap", layout_method="trivial")
draw(qc, style=IQXDebugging(), target=backend.target)

### Draw with a modified stylesheet

You can customize some parts of a preset stylesheet when you call it.

In [ ]:
from qiskit.visualization.timeline import draw, IQXStandard

my_style = {
    "formatter.general.fig_width": 16,
    "formatter.general.fig_unit_height": 1,
}
style = IQXStandard(**my_style)

# draw
draw(qc, style=style)

Similarly, you can create custom generator or layout functions and update an existing stylesheet with the custom functions. This way, you can control the most of the appearance of the output image without modifying the codebase of the scheduled circuit drawer.

## Qiskit Runtime support

While the timeline drawer built in to Qiskit is useful for static circuits, it might not accurately reflect the timing of [dynamic circuits](/docs/guides/classical-feedforward-and-control-flow) because of implicit operations such as broadcasting and branch determination. As part of dynamic circuit support, Qiskit Runtime returns the accurate circuit timing information inside the job results when requested.

### Enable timing data retrieval

To enable timing data retrieval, set the `scheduler_timing` flag to `True` when running the primitive job:

```python
sampler = SamplerV2(backend)
sampler.options.experimental = {
    "execution": {
        "scheduler_timing": True,
    },
}
```

### Access the circuit schedule data

Access the circuit's schedule data from the job's result metadata as follows.

```python
job_result: SamplerPubResult = job.result()
    circuit_schedule = job_result[0].metadata["compilation"]["scheduler_timing"]
    circuit_schedule_timing = circuit_schedule["timing"]
```

All of the data is returned under `compilation`. Therefore, you use that option, along the `timing` sub-argument to access specific data.

### Visualize the timings

You can display the output figure, save it to a file, or both.

- Run [`fig.show()`](https://plotly.com/python-api-reference/generated/plotly.io.show.html) display the figure.
- Run [`fig.write_image("<path.format>")`](https://plotly.com/python-api-reference/generated/plotly.io.write_image.html) download the figure.  For example: `fig.write_html("scheduler_timing.html")` or `fig.write_image("scheduler_timing.png")`.

### Arguments

The following arguments are used with timing visualizations.

- `compilation` - Used to retrieve values from circuit-timing related feature flags, in the format `result[0].metadata["compilation"][<feature-flag-name>]`.
- `timing` - Used to retrieve circuit timing data after it has been retrieved from the metadata.

### Understand the `circuit_schedule` input

The output from the job (input to `circuit_schedule`) is a list of strings, where each string represents a single instruction on some channel.

The instruction data is comma separated into `Branch`, `Instruction`, `Channel`, `T0`, `Duration`, and `Pulse` data types.

- `Branch` - Determines whether the instruction is in a control flow (then / else) or a main branch
- `Instruction` - The gate and the qubit to operate on
- `Channel` - The channel that is being assigned with the instruction (Qubit x / AWGRx_y / ...). Arbitrary Wave Generator Readout (AWGR) is used for readout channels communication for measuring qubits, as opposed to drive channels, which are for driving the qubits. The `X` and `Y` arguments correspond to the readout instrument ID and the qubit number, respectively.
- `T0` - The instruction start time within the complete schedule
- `Duration` - The instruction's duration
- `Pulse` - The type of pulse operation being used


Example:

```python
main,barrier,Qubit 0,7,0,barrier # A barrier on the main branch on qubit 0 at time 7 with 0 duration
main,reset_0,Qubit 0,7,64,play # A reset instruction on the main branch on qubit 0 at time 7 with duration 64 and a play operation
...
```

### End-to-end example

This example shows you how to enable the option, get it from the metadata, display it, and save it to a file.

First, set up the environment, define and transpile the circuits, and define and run the jobs.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.circuit import QuantumCircuit
from qiskit import transpile


service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Create a Bell circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw()

# Transpile the circuit for the given backend
transpiled_circuits = []
transpiled_circuit = transpile(qc, backend)
transpiled_circuits.append(transpiled_circuit)

# Generate samplers for backend targets
sampler = SamplerV2(backend)
sampler.options.experimental = {
    "execution": {"scheduler_timing": True},
}

# Submit jobs
sampler_job = sampler.run(transpiled_circuits)

print(
    f">>> {' Job ID:':<10}  {sampler_job.job_id()} ({sampler_job.status()})"
)

result = sampler_job.result()
print(f">>> {' Job:':<10} {sampler_job.job_id()} finished with:\n{result}")

Next, get the circuit schedule timing:

In [ ]:
# Get the circuit schedule timing
result[0].metadata["compilation"]["scheduler_timing"]["timing"]

Finally, you can visualize and save the timing:

In [ ]:
from qiskit_ibm_runtime.visualization import draw_circuit_schedule_timing

circuit_schedule = (
    result[0]
    .metadata["compilation"]["scheduler_timing"]["timing"]
    .split("\n")
)
fig = draw_circuit_schedule_timing(
    circuit_schedule=circuit_schedule,
    included_channels=None,
    filter_readout_channels=False,
    filter_barriers=False,
    width=1000,
)
fig.show()

## Next steps

<Admonition type="tip" title="Recommendations">
  -  Related information
</Admonition>